In [1]:
!pip install torchattacks

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.fft import dct, idct

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torchattacks
from torchattacks import PGD, FGSM, CW

from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
mnist_train = dsets.MNIST(root='./data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='./data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)
batch_size = 128

train_loader  = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                         batch_size=batch_size,
                                         shuffle=False)

In [17]:
model = models.resnet18()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, 10)
model = model.cuda()

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

atk = FGSM(model, eps=0)
atk1 = FGSM(model, eps=0.10)
atk2 = FGSM(model, eps=0.15)
atk3 = PGD(model, eps=0.10)
atk4 = PGD(model, eps=0.15)
atk5 = CW(model, c=0.10)
atk6 = CW(model, c=0.15)

num_epochs = 5

for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        # X = atk(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(batch_images.cuda())
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

"""model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = atk(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))"""

Freq_Scores = []
Avg = [0]*10
Count = [0]*10

for images, labels in test_loader:   

    for m in range(28):
        
        img1 = images.cpu().numpy()
        labels1 = labels.cpu()
        
        dct1 = dct(img1)
        dct2 = dct1.copy()
        mask = np.ma.masked_inside(np.abs(dct2),m,m+1).mask
        dct2[mask] = np.random.randint(low=-10, high=10)
        img2 = idct(dct2)

        img1 = torch.tensor(img1.reshape(-1,1,28,28)).cuda()
        img2 = torch.tensor(img2.reshape(-1,1,28,28)).cuda()
        out1 = model(img1)
        out2 = model(img2)

        Classes = torch.max(out1.data,1)[1].cpu()
        Diff = torch.sum(torch.abs(out1.data-out2.data),1).cpu()

        for i,val in enumerate(Classes):
            Avg[val] = (Avg[val]*Count[val] + Diff[i])/(Count[val]+1)
            Count[val]+=1 
        
        Freq_Scores.append(Avg.copy())

Freq_Scores = torch.tensor(Freq_Scores).view(-1,28,10)
Mean0 = torch.mean(Freq_Scores,0)

In [247]:
model = models.resnet18()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, 10)
model = model.cuda()

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# atk = PGD(model, eps=0.01)
atk1 = FGSM(model, eps=0.10)
atk2 = FGSM(model, eps=0.15)
atk3 = FGSM(model, eps=0.20)
atk4 = PGD(model, eps=0.10)
atk5 = PGD(model, eps=0.15)
atk6 = PGD(model, eps=0.20)

num_epochs = 5

for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = atk1(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = atk1(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Freq_Scores = []
Avg = [0]*10
Count = [0]*10

for images, labels in test_loader:   

    for m in range(28):
        
        img1 = images.cpu().numpy()
        labels1 = labels.cpu()
        
        dct1 = dct(img1)
        dct2 = dct1.copy()
        mask = np.ma.masked_inside(np.abs(dct2),m,m+1).mask
        dct2[mask] = np.random.randint(low=-10, high=10)
        img2 = idct(dct2)

        img1 = torch.tensor(img1.reshape(-1,1,28,28)).cuda()
        img2 = torch.tensor(img2.reshape(-1,1,28,28)).cuda()
        out1 = model(img1)
        out2 = model(img2)

        Classes = torch.max(out1.data,1)[1].cpu()
        Diff = torch.sum(torch.abs(out1.data-out2.data),1).cpu()

        for i,val in enumerate(Classes):
            Avg[val] = (Avg[val]*Count[val] + Diff[i])/(Count[val]+1)
            Count[val]+=1 
        
        Freq_Scores.append(Avg.copy())

Freq_Scores = torch.tensor(Freq_Scores).view(-1,28,10)
Mean1 = torch.mean(Freq_Scores,0)

Freq1 = np.zeros((28,28))

for images, labels in test_loader:
    
    attacked = atk1(images, labels).cuda()
    dummy = np.zeros((28,28))
    for img, atc in zip(images,attacked):
        true_tr = dct(img.cpu().numpy()).reshape(28,28)
        attacked_tr = dct(atc.cpu().numpy()).reshape(28,28)
        
        for i in range(28):
            for j in range(28):
                if true_tr[i][j]==0:
                    dummy[i][j]=0
                else:
                    dummy[i][j]=np.abs((attacked_tr[i][j]-true_tr[i][j])/true_tr[i][j])
    Freq1+=dummy

Freq1 = (Freq1-np.min(Freq1))/(np.max(Freq1)-np.min(Freq1))

In [248]:
model = models.resnet18()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, 10)
model = model.cuda()

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# atk = PGD(model, eps=0.01)
atk1 = FGSM(model, eps=0.10)
atk2 = FGSM(model, eps=0.15)
atk3 = FGSM(model, eps=0.20)
atk4 = PGD(model, eps=0.10)
atk5 = PGD(model, eps=0.15)
atk6 = PGD(model, eps=0.20)

num_epochs = 5

for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = atk2(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = atk2(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Freq_Scores = []
Avg = [0]*10
Count = [0]*10

for images, labels in test_loader:   

    for m in range(28):
        
        img1 = images.cpu().numpy()
        labels1 = labels.cpu()
        
        dct1 = dct(img1)
        dct2 = dct1.copy()
        mask = np.ma.masked_inside(np.abs(dct2),m,m+1).mask
        dct2[mask] = np.random.randint(low=-10, high=10)
        img2 = idct(dct2)

        img1 = torch.tensor(img1.reshape(-1,1,28,28)).cuda()
        img2 = torch.tensor(img2.reshape(-1,1,28,28)).cuda()
        out1 = model(img1)
        out2 = model(img2)

        Classes = torch.max(out1.data,1)[1].cpu()
        Diff = torch.sum(torch.abs(out1.data-out2.data),1).cpu()

        for i,val in enumerate(Classes):
            Avg[val] = (Avg[val]*Count[val] + Diff[i])/(Count[val]+1)
            Count[val]+=1 
        
        Freq_Scores.append(Avg.copy())

Freq_Scores = torch.tensor(Freq_Scores).view(-1,28,10)
Mean2 = torch.mean(Freq_Scores,0)

Freq2 = np.zeros((28,28))

for images, labels in test_loader:
    
    attacked = atk2(images, labels).cuda()
    dummy = np.zeros((28,28))
    for img, atc in zip(images,attacked):
        true_tr = dct(img.cpu().numpy()).reshape(28,28)
        attacked_tr = dct(atc.cpu().numpy()).reshape(28,28)
        
        for i in range(28):
            for j in range(28):
                if true_tr[i][j]==0:
                    dummy[i][j]=0
                else:
                    dummy[i][j]=np.abs((attacked_tr[i][j]-true_tr[i][j])/true_tr[i][j])
    Freq2+=dummy

Freq2 = (Freq2-np.min(Freq2))/(np.max(Freq2)-np.min(Freq2))

In [11]:
model = models.resnet18()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, 10)
model = model.cuda()

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# atk = PGD(model, eps=0.01)
atk1 = FGSM(model, eps=0.10)
atk2 = FGSM(model, eps=0.15)
atk3 = FGSM(model, eps=0.20)
atk4 = PGD(model, eps=0.10)
atk5 = PGD(model, eps=0.15)
atk6 = PGD(model, eps=0.20)

num_epochs = 3

for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = atk3(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

In [13]:
Freq_Scores = []
Avg = [0]*10
Count = [0]*10

for images, labels in test_loader:   

    for m in range(28):
        
        img1 = images.cpu().numpy()
        labels1 = labels.cpu()
        
        dct1 = dct(img1)
        dct2 = dct1.copy()
        mask = np.ma.masked_inside(np.abs(dct2),m,m+1).mask
        dct2[mask] = np.random.randint(low=-10, high=10)
        img2 = idct(dct2)

        img1 = torch.tensor(img1.reshape(-1,1,28,28)).cuda()
        img2 = torch.tensor(img2.reshape(-1,1,28,28)).cuda()
        out1 = model(img1)
        out2 = model(img2)

        Classes = torch.max(out1.data,1)[1].cpu()
        Diff = torch.sum(torch.abs(out1.data-out2.data),1).cpu()

        for i,val in enumerate(Classes):
            Avg[val] = (Avg[val]*Count[val] + Diff[i])/(Count[val]+1)
            Count[val]+=1 
        
        Freq_Scores.append(Avg.copy())

Freq_Scores = torch.tensor(Freq_Scores).view(-1,28,10)
Mean3 = torch.mean(Freq_Scores,0)

Freq3 = np.zeros((28,28))

for images, labels in test_loader:
    
    attacked = atk3(images, labels).cuda()
    dummy = np.zeros((28,28))
    for img, atc in zip(images,attacked):
        true_tr = dct(img.cpu().numpy()).reshape(28,28)
        attacked_tr = dct(atc.cpu().numpy()).reshape(28,28)
        
        for i in range(28):
            for j in range(28):
                if true_tr[i][j]==0:
                    dummy[i][j]=0
                else:
                    dummy[i][j]=np.abs((attacked_tr[i][j]-true_tr[i][j])/true_tr[i][j])
    Freq3+=dummy

Freq3 = (Freq3-np.min(Freq3))/(np.max(Freq3)-np.min(Freq3))

In [21]:
model = models.resnet18()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, 10)
model = model.cuda()

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# atk = PGD(model, eps=0.01)
atk1 = FGSM(model, eps=0.10)
atk2 = FGSM(model, eps=0.15)
atk3 = FGSM(model, eps=0.20)
atk4 = PGD(model, eps=0.10)
atk5 = PGD(model, eps=0.15)
atk6 = PGD(model, eps=0.20)

num_epochs = 2

for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = atk4(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = atk4(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Freq_Scores = []
Avg = [0]*10
Count = [0]*10

for images, labels in test_loader:   

    for m in range(28):
        
        img1 = images.cpu().numpy()
        labels1 = labels.cpu()
        
        dct1 = dct(img1)
        dct2 = dct1.copy()
        mask = np.ma.masked_inside(np.abs(dct2),m,m+1).mask
        dct2[mask] = np.random.randint(low=-10, high=10)
        img2 = idct(dct2)

        img1 = torch.tensor(img1.reshape(-1,1,28,28)).cuda()
        img2 = torch.tensor(img2.reshape(-1,1,28,28)).cuda()
        out1 = model(img1)
        out2 = model(img2)

        Classes = torch.max(out1.data,1)[1].cpu()
        Diff = torch.sum(torch.abs(out1.data-out2.data),1).cpu()

        for i,val in enumerate(Classes):
            Avg[val] = (Avg[val]*Count[val] + Diff[i])/(Count[val]+1)
            Count[val]+=1 
        
        Freq_Scores.append(Avg.copy())

Freq_Scores = torch.tensor(Freq_Scores).view(-1,28,10)
Mean4 = torch.mean(Freq_Scores,0)

Freq4 = np.zeros((28,28))

for images, labels in test_loader:
    
    attacked = atk4(images, labels).cuda()
    dummy = np.zeros((28,28))
    for img, atc in zip(images,attacked):
        true_tr = dct(img.cpu().numpy()).reshape(28,28)
        attacked_tr = dct(atc.cpu().numpy()).reshape(28,28)
        
        for i in range(28):
            for j in range(28):
                if true_tr[i][j]==0:
                    dummy[i][j]=0
                else:
                    dummy[i][j]=np.abs((attacked_tr[i][j]-true_tr[i][j])/true_tr[i][j])
    Freq4+=dummy

Freq4 = (Freq4-np.min(Freq4))/(np.max(Freq4)-np.min(Freq4))

In [16]:
model = models.resnet18()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, 10)
model = model.cuda()

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# atk = PGD(model, eps=0.01)
atk1 = FGSM(model, eps=0.10)
atk2 = FGSM(model, eps=0.15)
atk3 = FGSM(model, eps=0.20)
atk4 = PGD(model, eps=0.10)
atk5 = PGD(model, eps=0.15)
atk6 = PGD(model, eps=0.20)

num_epochs = 2

for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = atk5(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = atk5(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Freq_Scores = []
Avg = [0]*10
Count = [0]*10

for images, labels in test_loader:   

    for m in range(28):
        
        img1 = images.cpu().numpy()
        labels1 = labels.cpu()
        
        dct1 = dct(img1)
        dct2 = dct1.copy()
        mask = np.ma.masked_inside(np.abs(dct2),m,m+1).mask
        dct2[mask] = np.random.randint(low=-10, high=10)
        img2 = idct(dct2)

        img1 = torch.tensor(img1.reshape(-1,1,28,28)).cuda()
        img2 = torch.tensor(img2.reshape(-1,1,28,28)).cuda()
        out1 = model(img1)
        out2 = model(img2)

        Classes = torch.max(out1.data,1)[1].cpu()
        Diff = torch.sum(torch.abs(out1.data-out2.data),1).cpu()

        for i,val in enumerate(Classes):
            Avg[val] = (Avg[val]*Count[val] + Diff[i])/(Count[val]+1)
            Count[val]+=1 
        
        Freq_Scores.append(Avg.copy())

Freq_Scores = torch.tensor(Freq_Scores).view(-1,28,10)
Mean5 = torch.mean(Freq_Scores,0)

Freq5 = np.zeros((28,28))

for images, labels in test_loader:
    
    attacked = atk5(images, labels).cuda()
    dummy = np.zeros((28,28))
    for img, atc in zip(images,attacked):
        true_tr = dct(img.cpu().numpy()).reshape(28,28)
        attacked_tr = dct(atc.cpu().numpy()).reshape(28,28)
        
        for i in range(28):
            for j in range(28):
                if true_tr[i][j]==0:
                    dummy[i][j]=0
                else:
                    dummy[i][j]=np.abs((attacked_tr[i][j]-true_tr[i][j])/true_tr[i][j])
    Freq5+=dummy

Freq5 = (Freq5-np.min(Freq5))/(np.max(Freq5)-np.min(Freq5))

In [14]:
model = models.resnet18()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, 10)
model = model.cuda()

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# atk = PGD(model, eps=0.01)
atk1 = FGSM(model, eps=0.10)
atk2 = FGSM(model, eps=0.15)
atk3 = FGSM(model, eps=0.20)
atk4 = PGD(model, eps=0.10)
atk5 = PGD(model, c=0.15)
atk6 = PGD(model, eps=0.20)

num_epochs = 3

for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = atk6(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = atk6(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Freq_Scores = []
Avg = [0]*10
Count = [0]*10

for images, labels in test_loader:   

    for m in range(28):
        
        img1 = images.cpu().numpy()
        labels1 = labels.cpu()
        
        dct1 = dct(img1)
        dct2 = dct1.copy()
        mask = np.ma.masked_inside(np.abs(dct2),m,m+1).mask
        dct2[mask] = np.random.randint(low=-10, high=10)
        img2 = idct(dct2)

        img1 = torch.tensor(img1.reshape(-1,1,28,28)).cuda()
        img2 = torch.tensor(img2.reshape(-1,1,28,28)).cuda()
        out1 = model(img1)
        out2 = model(img2)

        Classes = torch.max(out1.data,1)[1].cpu()
        Diff = torch.sum(torch.abs(out1.data-out2.data),1).cpu()

        for i,val in enumerate(Classes):
            Avg[val] = (Avg[val]*Count[val] + Diff[i])/(Count[val]+1)
            Count[val]+=1 
        
        Freq_Scores.append(Avg.copy())

Freq_Scores = torch.tensor(Freq_Scores).view(-1,28,10)
Mean6 = torch.mean(Freq_Scores,0)

Freq6 = np.zeros((28,28))

for images, labels in test_loader:
    
    attacked = atk6(images, labels).cuda()
    dummy = np.zeros((28,28))
    for img, atc in zip(images,attacked):
        true_tr = dct(img.cpu().numpy()).reshape(28,28)
        attacked_tr = dct(atc.cpu().numpy()).reshape(28,28)
        
        for i in range(28):
            for j in range(28):
                if true_tr[i][j]==0:
                    dummy[i][j]=0
                else:
                    dummy[i][j]=np.abs((attacked_tr[i][j]-true_tr[i][j])/true_tr[i][j])
    Freq6+=dummy

Freq6 = (Freq6-np.min(Freq6))/(np.max(Freq6)-np.min(Freq6))

In [33]:
fig, axs = plt.subplots(2, 5, figsize=(20,7))
#axs[0,0].plot(Mean0[:,0], label='Natural')
#axs[0,1].plot(Mean0[:,1], label='Natural')
#axs[0,2].plot(Mean0[:,2], label='Natural')
#axs[0,3].plot(Mean0[:,3], label='Natural')
#axs[0,4].plot(Mean0[:,4], label='Natural')
#axs[1,0].plot(Mean0[:,5], label='Natural')
#axs[1,1].plot(Mean0[:,6], label='Natural')
#axs[1,2].plot(Mean0[:,7], label='Natural')
#axs[1,3].plot(Mean0[:,8], label='Natural')
#axs[1,4].plot(Mean0[:,9], label='Natural')
#axs[0,0].plot(Mean4[:,0], label='PGD eps=0.10')
#axs[0,0].plot(Mean5[:,0], label='PGD eps=0.15')
axs[0,0].plot(Mean6[:,0], label='PGD eps=0.20')
axs[0,0].legend()
#axs[0,1].plot(Mean4[:,1], label='PGD eps=0.10')
#axs[0,1].plot(Mean5[:,1], label='PGD eps=0.15')
axs[0,1].plot(Mean6[:,1], label='PGD eps=0.20')
axs[0,1].legend()
#axs[0,2].plot(Mean4[:,2], label='PGD eps=0.10')
#axs[0,2].plot(Mean5[:,2], label='PGD eps=0.15')
axs[0,2].plot(Mean6[:,2], label='PGD eps=0.20')
axs[0,2].legend()
#axs[0,3].plot(Mean4[:,3], label='PGD eps=0.10')
#axs[0,3].plot(Mean5[:,3], label='PGD eps=0.15')
axs[0,3].plot(Mean6[:,3], label='PGD eps=0.20')
axs[0,3].legend()
#axs[0,4].plot(Mean4[:,4], label='PGD eps=0.10')
#axs[0,4].plot(Mean5[:,4], label='PGD eps=0.15')
axs[0,4].plot(Mean6[:,4], label='PGD eps=0.20')
axs[0,4].legend()
#axs[1,0].plot(Mean4[:,5], label='PGD eps=0.10')
#axs[1,0].plot(Mean5[:,5], label='PGD eps=0.15')
axs[1,0].plot(Mean6[:,5], label='PGD eps=0.20')
axs[1,0].legend()
#axs[1,1].plot(Mean4[:,6], label='PGD eps=0.10')
#axs[1,1].plot(Mean5[:,6], label='PGD eps=0.15')
axs[1,1].plot(Mean6[:,6], label='PGD eps=0.20')
axs[1,1].legend()
#axs[1,2].plot(Mean4[:,7], label='PGD eps=0.10')
#axs[1,2].plot(Mean5[:,7], label='PGD eps=0.15')
axs[1,2].plot(Mean6[:,7], label='PGD eps=0.20')
axs[1,2].legend()
#axs[1,3].plot(Mean4[:,8], label='PGD eps=0.10')
#axs[1,3].plot(Mean5[:,8], label='PGD eps=0.15')
axs[1,3].plot(Mean6[:,8], label='PGD eps=0.20')
axs[1,3].legend()
#axs[1,4].plot(Mean4[:,9], label='PGD eps=0.10')
#axs[1,4].plot(Mean5[:,9], label='PGD eps=0.15')
axs[1,4].plot(Mean6[:,9], label='PGD eps=0.20')
axs[1,4].legend()
axs[0,0].set_title('0')
axs[0,1].set_title('1')
axs[0,2].set_title('2')
axs[0,3].set_title('3')
axs[0,4].set_title('4')
axs[1,0].set_title('5')
axs[1,1].set_title('6')
axs[1,2].set_title('7')
axs[1,3].set_title('8')
axs[1,4].set_title('9')
plt.savefig('PGD_0.20_FAS_Scores.png')

In [24]:
fig, axs = plt.subplots(1, 3, figsize=(15,5))
m1 = axs[0].imshow(Freq4, label='PGD eps=0.10')
m2 = axs[1].imshow(Freq5, label='PGD eps=0.15')
m3 = axs[2].imshow(Freq6, label='PGD eps=0.20')
#m4 = axs[1,1].imshow(Freq2, label='PGD eps=0.15')

axs[0].set_title("PGD eps=0.10")
axs[1].set_title("PGD eps=0.15")
axs[2].set_title("PGD eps=0.20")
divider = make_axes_locatable(axs[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(m1, cax=cax, orientation='vertical')
divider = make_axes_locatable(axs[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(m2, cax=cax, orientation='vertical')
divider = make_axes_locatable(axs[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(m3, cax=cax, orientation='vertical')

plt.savefig("PGD_RCT_Maps.png")